In [2]:
import unicodedata
import re
DEVICE_SLUGS = ["phone", "laptop", "tablet"] 
KNOWN_CATEGORIES = {
    "dien thoai": "phone", "may tinh bang": "tablet", "laptop": "laptop",
    "pin du phong": "pinduphong", "cap sac": "capsac", "tai nghe": "tainghe"
}

def normalize_search_term(query: str) -> str:
    """Chuẩn hóa chuỗi tìm kiếm: hạ chữ, bỏ dấu, và thay thế từ viết tắt."""
    text = query.lower().strip()
    
    # 1. Bỏ dấu tiếng Việt
    text = unicodedata.normalize('NFD', text)
    text = re.sub(r'[\u0300-\u036f]', '', text)
    text = text.replace('đ', 'd')
    
    # 2. Thay thế Từ viết tắt phổ biến
    replacements = {
        r'\bdt\b': 'dien thoai', 
        r'\blt\b': 'laptop',
        r'\bss\b': 'samsung',
        r'\bip\b': 'iphone',
        r'\bpin dp\b': 'pin du phong'
    }
    for old, new in replacements.items():
        text = re.sub(old, new, text)
        
    # Loại bỏ ký tự đặc biệt, nhưng giữ lại '/','-', '.' và '+' 
    text = re.sub(r'[^a-z0-9\s/+-.]', ' ', text)
    
    # Rút gọn khoảng trắng thừa
    return " ".join(text.split())

normalize_search_term("dung lượng")

'dung luong'

In [ ]:
# app/routers/product.py
from fastapi import APIRouter, HTTPException, Query
from typing import List, Optional, Dict
from datetime import datetime
from app.core.config import SessionLocal
from app.models.product import Product, ProductImage, Specification, Abs
from app.schemas.product import (
    ProductSchema, AddProductSchema, AttributeSchema,
    ImagesSchema, PromotionSchema, AbsProduct, OutPutAbs, ProductSearchResult
)
from sqlalchemy import text
router = APIRouter()

# -------------------- PRODUCT --------------------
@router.get("/product", response_model=List[ProductSchema])
def get_product(type: Optional[str] = None):
    db = SessionLocal()
    try:
        if type:
            products = db.query(Product).filter(Product.phanloai == type).all()
        else:
            products = db.query(Product).all()
        return products
    finally:
        db.close()


@router.get("/product_id", response_model=AddProductSchema)
def get_product_by_id(id: Optional[int] = None):
    db = SessionLocal()
    try:
        product = db.query(Product).filter(Product.id == id).first()
        if not product:
            raise HTTPException(404, "Không tìm thấy sản phẩm")

        # Lấy specifications theo product_id
        attrs = db.query(Specification).filter(Specification.product_id == id).all()
        attributes = [
            AttributeSchema(key=attr.spec, value=attr.info, loai_cau_hinh=attr.loai_cau_hinh)
            for attr in attrs
        ]

        # Lấy images
        imgs = db.query(ProductImage.img).filter(ProductImage.product_id == id).all()
        images = [ImagesSchema(img=img.img) for img in imgs]

        # Lấy promotion hiện tại (nếu có)
        now = datetime.now()
        present_abs = db.query(Abs).filter(
            Abs.start_time <= now,
            Abs.end_time >= now,
            Abs.product_id == id
        ).first()

        promotion = None
        if present_abs:
            promotion = PromotionSchema(
                percent_abs=present_abs.percent_abs,
                start_time=present_abs.start_time,
                end_time=present_abs.end_time
            )

        # Gói dữ liệu trả về
        db_product = AddProductSchema(
            name=product.name,
            phanloai=product.phanloai,
            price=product.price,
            thumb=product.thumb,
            main_image=product.main_image,
            brand=product.brand,
            release_date=product.release_date,
            attributes=attributes,
            images=images,
            promotion=promotion
        )
        return db_product
    finally:
        db.close()


@router.post("/product")
def add_product(product: AddProductSchema):
    db = SessionLocal()
    try:
        prod = db.query(Product).filter(Product.name == product.name).first()
        if prod:
            raise HTTPException(404, detail="Product already exists")

        db_product = Product(
            name=product.name,
            phanloai=product.phanloai,
            price=product.price,
            thumb=product.thumb,
            main_image=product.main_image,
            brand=product.brand,
            release_date=product.release_date,
        )
        db.add(db_product)
        db.commit()
        db.refresh(db_product)

        # thêm attributes
        for attr in product.attributes:
            db_attr = Specification(
                spec=attr.key,
                info=attr.value,
                loai_cau_hinh=attr.loai_cau_hinh,
                product_id=db_product.id
            )
            db.add(db_attr)
        db.commit()

        # thêm images
        for img in product.images:
            db_img = ProductImage(product_id=db_product.id, img=img.img)
            db.add(db_img)
        db.commit()

        # thêm promotion
        pro = product.promotion
        if pro:
            db_promo = Abs(
                product_id=db_product.id,
                percent_abs=pro.percent_abs,
                start_time=pro.start_time,
                end_time=pro.end_time
            )
            db.add(db_promo)
            db.commit()

        return {"message": "Product is added"}
    finally:
        db.close()


@router.put("/product/{product_id}")
def update_product(product_id: int, payload: AddProductSchema):
    db = SessionLocal()
    try:
        product = db.query(Product).filter(Product.id == product_id).first()
        if not product:
            raise HTTPException(status_code=404, detail="Product not found")

        # Cập nhật các field cơ bản
        product.name = payload.name
        product.phanloai = payload.phanloai
        product.price = payload.price
        product.brand = payload.brand
        product.release_date = payload.release_date
        product.thumb = payload.thumb
        product.main_image = payload.main_image

        # Xóa attributes cũ -> thêm mới
        db.query(Specification).filter(Specification.product_id == product_id).delete()
        for attr in payload.attributes:
            spec = Specification(
                product_id=product_id,
                spec=attr.key,
                info=attr.value,
                loai_cau_hinh=attr.loai_cau_hinh
            )
            db.add(spec)

        # Xóa images cũ -> thêm mới
        db.query(ProductImage).filter(ProductImage.product_id == product_id).delete()
        for img in payload.images:
            image = ProductImage(product_id=product_id, img=img.img)
            db.add(image)

        # Cập nhật promotion
        db.query(Abs).filter(Abs.product_id == product_id).delete()
        if payload.promotion:
            promo = Abs(
                product_id=product_id,
                percent_abs=payload.promotion.percent_abs,
                start_time=payload.promotion.start_time,
                end_time=payload.promotion.end_time
            )
            db.add(promo)

        db.commit()
        db.refresh(product)
        return {"message": "Product updated successfully", "product": product}
    finally:
        db.close()


@router.delete("/product")
def delete_product(product_id: int):
    db = SessionLocal()
    try:
        product_c = db.query(Product).filter(Product.id == product_id).first()
        if product_c:
            db.delete(product_c)
            db.commit()
            return {"message": "Product was deleted"}
        raise HTTPException(404, "Product not found")
    finally:
        db.close()

# -------------------- ABS --------------------
@router.post("/abs", response_model=AbsProduct)
def push_abs(abs: AbsProduct):
    db = SessionLocal()
    try:
        newabs = Abs(**abs.model_dump())
        now = datetime.now()

        existabs = db.query(Abs).filter(
            Abs.start_time <= now,
            Abs.end_time >= now,
            Abs.product_id == newabs.product_id
        ).first()
        if existabs:
            raise HTTPException(400, "Product is existed in another Abs")

        db.add(newabs)
        db.commit()
        db.refresh(newabs)
        return newabs
    finally:
        db.close()


@router.get("/abs", response_model=List[OutPutAbs])
def get_abs(type: Optional[str] = None, page: int = 1, limit: int = 20, show_abs: bool = False):
    db = SessionLocal()
    try:
        now = datetime.now()
        present_abs = (
            db.query(Product, Abs)
            .outerjoin(
                Abs,
                (Abs.product_id == Product.id) &
                (Abs.start_time <= now) &
                (Abs.end_time >= now)
            )
        )

        if type == "phukien":
            present_abs = present_abs.filter(~Product.phanloai.in_(["phone", "laptop", "tablet"]))
        elif type:
            present_abs = present_abs.filter(Product.phanloai == type)

        total = present_abs.count()

        # phân trang
        if not show_abs:
            present_abs = present_abs.offset((page - 1) * limit).limit(limit).all()
        result = []


        for product_obj, abs_obj in present_abs:
            if product_obj.phanloai == "laptop" and "(" in product_obj.name:
                product_obj.name = product_obj.name.split("(")[0]

            result.append(
                OutPutAbs(
                    id=product_obj.id,
                    name=product_obj.name,
                    price=product_obj.price,
                    thumb=product_obj.thumb,
                    main_image=product_obj.main_image,
                    phanloai=product_obj.phanloai,
                    brand=product_obj.brand,
                    release_date=product_obj.release_date,
                    percent_abs=abs_obj.percent_abs if abs_obj else 0,
                    start_time=abs_obj.start_time if abs_obj else None,
                    end_time=abs_obj.end_time if abs_obj else None,
                )
            )
        return result
    finally:
        db.close()

import unicodedata
import re
DEVICE_SLUGS = ["phone", "laptop", "tablet"] 
KNOWN_CATEGORIES = {
    "dien thoai": "phone", "may tinh bang": "tablet", "laptop": "laptop",
    "pin du phong": "pinduphong", "cap sac": "capsac", "tai nghe": "tainghe"
}

def normalize_search_term(query: str) -> str:
    """Chuẩn hóa chuỗi tìm kiếm: hạ chữ, bỏ dấu, và thay thế từ viết tắt."""
    text = query.lower().strip()
    
    # 1. Bỏ dấu tiếng Việt
    text = unicodedata.normalize('NFD', text)
    text = re.sub(r'[\u0300-\u036f]', '', text)
    text = text.replace('đ', 'd')
    
    # 2. Thay thế Từ viết tắt phổ biến
    replacements = {
        r'\bdt\b': 'dien thoai', 
        r'\blt\b': 'laptop',
        r'\bss\b': 'samsung',
        r'\bip\b': 'iphone',
        r'\bpin dp\b': 'pin du phong'
    }
    for old, new in replacements.items():
        text = re.sub(old, new, text)
        
    # Loại bỏ ký tự đặc biệt, nhưng giữ lại '/','-', '.' và '+' 
    text = re.sub(r'[^a-z0-9\s/+-.]', ' ', text)
    
    # Rút gọn khoảng trắng thừa
    return " ".join(text.split())
def parse_query_to_conditions(query: str) -> dict:
    """
    Phân tích truy vấn, trích xuất thông số kỹ thuật (RAM, Pin, Lưu trữ) 
    và ánh xạ ngược thành cụm từ chính xác (Exact Match) và từ khóa mở rộng (Broad Match).
    """
    conditions: Dict[str] = {}
    remaining_query = query
    
    # 1. Tìm và loại bỏ Loại sản phẩm
    conditions['phanloai'] = None
    for cat_name, cat_slug in KNOWN_CATEGORIES.items():
        if re.search(r'\b' + re.escape(cat_name) + r'\b', remaining_query):
            conditions['phanloai'] = cat_slug
            remaining_query = re.sub(r'\b' + re.escape(cat_name) + r'\b', ' ', remaining_query).strip()
            break
            
    text_filter_parts = []
    broad_search_terms = [] 
    

    is_device = conditions['phanloai'] in DEVICE_SLUGS
    

    ram_match = re.search(r'\b(\d+)\s*(gb)?\s*ram\b|\bram\s*(\d+)\s*(gb)?\b', remaining_query)
    if ram_match:
        ram_gb = int(ram_match.group(1) or ram_match.group(3))
        text_filter_parts.append(f'"RAM: {ram_gb} GB"') 
        broad_search_terms.extend([str(ram_gb), "ram", "gb"])
        remaining_query = re.sub(r'\b' + re.escape(ram_match.group(0)) + r'\b', ' ', remaining_query)


    storage_pattern1 = r'\b(\d+)\s*(gb|g)?\s*(dung\s*luong|luu\s*tru|bo\s*nho|storage|rom)\b'

    storage_pattern2 = r'\b(dung\s*luong|luu\s*tru|bo\s*nho|storage|rom)\s*(\d+)\s*(gb|g)?\b'
    
    storage_match = re.search(storage_pattern1, remaining_query) or re.search(storage_pattern2, remaining_query)
    
    if storage_match:
        if re.search(storage_pattern1, remaining_query):
            # Pattern 1: group(1) chứa số
            storage_gb = int(storage_match.group(1))
        else:
            # Pattern 2: group(2) chứa số
            storage_gb = int(storage_match.group(2))
            
        text_filter_parts.append(f'"Dung lượng lưu trữ: {storage_gb} GB"')
        broad_search_terms.extend([str(storage_gb), "bo", "nho", "dung", "luong", "gb"])
        remaining_query = re.sub(r'\b' + re.escape(storage_match.group(0)) + r'\b', ' ', remaining_query)

    pin_match = re.search(r'\b(\d+)\s*(mah)?\s*pin\b|\bpin\s*(\d+)\s*(mah)?\b', remaining_query)
    if pin_match:
        pin_mah = int(pin_match.group(1) or pin_match.group(3))
        text_filter_parts.append(f'"Dung lượng pin: {pin_mah} mAh"')
        broad_search_terms.extend([str(pin_mah), "mah", "pin"])
        remaining_query = re.sub(r'\b' + re.escape(pin_match.group(0)) + r'\b', ' ', remaining_query)

    cleaned_text_search = " ".join(remaining_query.split())
    broad_search_terms.extend(cleaned_text_search.split())
    

    unique_broad_terms = set(broad_search_terms)
    

    exact_terms_sql = [f'{term}' for term in text_filter_parts] 
    

    broad_terms_sql = [
        f'{word}' 
        for word in unique_broad_terms 
        if len(word) > 1 and word.isalnum()
    ]
    
    final_text_search_query = " ".join(exact_terms_sql + broad_terms_sql)

    if final_text_search_query:
        conditions['text_search'] = final_text_search_query
        
    return conditions

@router.get("/search/", response_model=List[ProductSearchResult])
def search_products(
    q: str = Query(..., min_length=2, description="Từ khóa tìm kiếm"),
):
    db = SessionLocal()
    
    normalized_q = normalize_search_term(q)
    if (normalized_q == "bo nho"):
        normalized_q = "dung luong"
    conditions = parse_query_to_conditions(normalized_q)
    

    where_clauses = []
    params = {}
    
    if conditions.get('phanloai'):
        where_clauses.append("phanloai = :phanloai")
        params['phanloai'] = conditions['phanloai']
        

    final_text_query = conditions.get('text_search', '')

    score_clause = "0 AS relevance_score"
    order_by_clause = "ORDER BY id DESC"
    
    if final_text_query:
        fulltext_match_clause = "MATCH(name, phanloai_vi, brand, cauhinh_daydu) AGAINST(:text_query IN BOOLEAN MODE)"
        

        where_clauses.append(fulltext_match_clause)
        params['text_query'] = final_text_query
        

        score_clause = f"({fulltext_match_clause}) * 5 AS relevance_score"
        order_by_clause = "ORDER BY relevance_score DESC"
        
    if not where_clauses:
        db.close()
        return []

    where_sql = " AND ".join(where_clauses)

    sql_query = text(f"""
        SELECT 
            id, name, price, phanloai_vi, phanloai, brand, cauhinh_daydu, -- Chọn các cột cần thiết
            {score_clause}
        FROM product_search
        WHERE {where_sql}
        {order_by_clause}
    """)

    try:
        results = db.execute(sql_query, params).fetchall()
        return results
    except Exception as e:
        print(f"Lỗi truy vấn SQL: {e}")
        # Cung cấp chi tiết lỗi để tiện debug
        print(f"Truy vấn SQL lỗi: {sql_query.string}")
        print(f"Tham số: {params}")
        raise e
    finally:
        db.close()


Input: Điện thoại dung luong 64GB
Normalized: dien thoai dung luong 64gb
Result: {'phanloai': 'phone', 'text_search': '"Dung lượng lưu trữ: 64 GB" bo gb dung luong nho 64'}
--------------------------------------------------
Input: dien thoai 128gb bo nho
Normalized: dien thoai 128gb bo nho
Result: {'phanloai': 'phone', 'text_search': '"Dung lượng lưu trữ: 128 GB" gb luong dung 128 bo nho'}
--------------------------------------------------
Input: laptop luu tru 256g
Normalized: laptop luu tru 256g
Result: {'phanloai': 'laptop', 'text_search': '"Dung lượng lưu trữ: 256 GB" gb 256 dung luong bo nho'}
--------------------------------------------------
Input: 512gb storage phone
Normalized: 512gb storage phone
Result: {'phanloai': None, 'text_search': '"Dung lượng lưu trữ: 512 GB" phone gb 512 dung luong bo nho'}
--------------------------------------------------
Input: bo nho 32gb
Normalized: bo nho 32gb
Result: {'phanloai': None, 'text_search': '"Dung lượng lưu trữ: 32 GB" gb 32 dung luo